# Импорты

# Выпуклая оболочка в $n$-мерном пространстве

Нам необходимо реализовать алгоритм quick hull в $n$-мерном пространстве.


# Геометрические понятия

* Симплекс (точнее, $n$-симплекс, где число $n$ называется размерностью симплекса) — это выпуклая оболочка $n+1$ точки аффинного пространства (размерности n или больше), которые предполагаются аффинно независимыми (то есть не лежат в подпространстве размерности $n–1$). Эти точки называются вершинами симплекса.
* Многогранник (син. политоп, многогранник, полиэдр, англ. polytope, polyhedron) определяется как минимум $n+1$ аффинно независимой точкой (вершиной); симплекс (англ. simplicial polytope) — простейший случай $n$-мерного тела, многогранники с меньшим числом вершин обязательно будут иметь нулевой $n$-мерный объём.
* Параллелотоп (англ. parallelotope) — обобщение плоского параллелограмма и объёмного параллелепипеда. Для симплекса можно построить $n+1$ соответственных параллелотопов (все они будут иметь одинаковый объём, равный $n!$ объёмам симплекса), взяв в качестве образующих (векторов) параллелотопа вектора, выходящие из одной фиксированной вершины в остальные $n$ вершины.
* Симплициальная грань (англ. simplicial facet) определяется $n$ аффинно независимыми точками (вершинами). Далее речь будет идти только о симплициальных объектах (кроме выпуклого многогранника), так что определение «симплициальный» я буду опускать. Многие утверждения в дальнейшем будут справедливы только для невырожденных (все точки попарно различны) случаев симплициальных геометрических объектов.
* Ребро (англ. ridge) определяется как пересечение двух граней и имеет $n–1$ вершину. Две грани могут иметь только одно общее ребро. Одна грань, таким образом, может иметь $n$ соседей через рёбра.
* Пик (англ. peak) определяется $n–1$ точками. Здесь интуитивная аналогия с трёхмерным пространством может поплыть, так как понятие пика и вершины не совпадают, но такими объектами мы не будем оперировать. Грань может иметь любое количество соседних граней через пик, отсюда можно сделать вывод, что хранить граф смежности граней через пики накладно и по памяти и по затратам на поддержку актуальности структур данных при каких-либо преобразованиях.

Грань многогранника, её границы, границы её границ и т.д. на английском именуются face.  Английский термин facet — это  наблюдаемая грань. Прямая (англ. straight line) — это всегда одномерная сущность. Точку (вершину) можно в этой градации считать нуль-мерной.

## Гиперобъём

Любой рассмотреный объект из ряда симплекс, грань, ребро, пик и т.д. ограничены в соответственном подпространстве. «Количество места», которое такой объект занимает, можно измерить/определить/задать. Для одномерной прямой мера называется длиной; для двумерной плоскости, мера называется площадью; для трёхмерного тела — объём. Понятие можно обобщить, назвав D-мерным объёмом или D-гиперобъёмом. Для симплекса, образованного попарно различными точками $(p_1, p_2,...,p_{n-1}, p_{n+1})$ (порядок перечисления точек важен), гиперобъём вычислить можно так:
$p_i = (p_i^1, p_i^2,...,p_i^n), i \in \{1,2,...n,n+1\}$

$\Delta{p_i} = (\Delta{p_i^1}, \Delta{p_i^2},...,\Delta{p_i^n}) = p_i - p_{n+1}, i \in \{1,2,...n,n+1\}$

$volume(conv(p_1,p_2, ..., p_n, p{n+1})) = \frac{1}{n!} \begin{bmatrix}
 \Delta{p_1^1}&\Delta{p_1^2}&\dots&\Delta{p_1^n}\\ 
 \Delta{p_2^1}&\Delta{p_2^2}&\dots&\Delta{p_2^n}\\
 \vdots&\vdots&\ddots&\vdots\\
 \Delta{p_n^1}&\Delta{p_n^2}&\dots&\Delta{p_n^n}\\ 
\end{bmatrix}$

Здесь мы выписали координаты векторов в строки. Аналогичные формулы и рассуждения можно привести и для векторов-столбцов (т.е. если мы транспонируем все матрицы, то на результат и выводы это не повлияет).
Делитель детерминанта в формуле выше — это количество симплексов (все они имеют одинаковый объём), на который можно разбить параллелотоп $\{\sum_{i=1}^{n} \overrightarrow{\Delta{p_i}}|\; 0 \leq c_i \leq 1\}$, построенный на векторах $ \overrightarrow{\Delta{p_i}}$. Соответственно сам детерминант — это гиперобъём параллелотопа. Интересующимся основаниями, лежащими в основе этого утверждения, рекомендую прочитать про определитель матрицы Грама и о его геометрической интерпретации.
Следует отметить, что данная мера для «объёмных» объектов будет иметь ненулевое значение, а также может быть как положительной, так и отрицательной величиной. Понять смысл знака нетрудно из следующих соображений: при обмене местами двух точек симплекса мы получаем смену знака детерминанта; порядок точек — это «лево- и право- ориентированность» симплекса. На плоскости это несложно себе представить: если стороны треугольника $(1,0),(0,1),(0,0)$ перечислены против часовой стрелки $((1,0),(0,1),(0,0))$, то определитель положителен 
$\begin{bmatrix}
1&0\\
0&1
\end{bmatrix} > 0$, иначе $((0,1),(1,0),(0,0))$ — отрицателен $\begin{bmatrix}
0&1\\
1&0
\end{bmatrix} < 0$. Для тетраэдра знак будет зависеть от того, в каком порядке (по часовой стрелке или против) первые 3 точки будут перечисляться при взгляде на них из последней. Таким образом, в дальнейшем, при задании геометрических объектов, мы должны принять, что порядок перечисления точек должен быть фиксированным.
Множитель перед детерминантом мы можем опустить, так как в алгоритме будет использоваться лишь информация о знаке величины ориентированного гиперобъёма.
Детерминант равен нулю, если хотя бы две строки линейно зависимы (помним, что точки попарно различны, то есть ни одна строка не нулевая). Несложно проверить соответствие приведённых выше рассуждений об аффинно зависимых точках и линейно зависимых векторах, соответствующих им.
На абсолютное значение детерминанта не будет влиять то, какую именно точку вычитаем при переходе к векторам, — только на знак. Следует вычитать всегда последнюю точку из первых, иначе для одинаково ориентированных аналогичных объектов, используемых в дальнейшем, знак меры будет разным в чётных и нечётных размерностях.
Что же делать с мерой для объектов, вложенных в пространство слишком большой размерности, например, с гранями? Если мы сконструируем матрицу так же, как это показано выше, то она будет прямоугольная. Детерминант от такой матрицы взять не получится. Оказывается формулу можно обобщить (это обобщение связано с формулой Бине-Коши), используя всё ту же матрицу составленную из векторов-строк:
$p_i = (p_i^1, p_i^2,...,p_i^n), i \in \{1,2,...m-1,m\}, m \leq n$

$P = (p_1, p_2, \dots, p_m)$

$\Delta{p_i} = (\Delta{p_i^1}, \Delta{p_i^2},...,\Delta{p_i^n}) = p_i - p_{m}, i \in \{1,2,...m-1\}$

$ G(p_1, p_2, p_{m-1}, p_m) =
\begin{bmatrix}
 \Delta{p_1^1}&\Delta{p_1^2}&\dots&\Delta{p_1^n}\\ 
 \Delta{p_2^1}&\Delta{p_2^2}&\dots&\Delta{p_2^n}\\
 \vdots&\vdots&\ddots&\vdots\\
 \Delta{p_{m-1}^1}&\Delta{p_{m-1}^2}&\dots&\Delta{p_{m-1}^n}\\ 
\end{bmatrix} $
$ measure(conv(p_1, p_2, \dots, p_{m-1}, p_m)) = \frac{1}{(m-1)!}\cdot\sqrt{det \mid\mid G(P)\cdot G^{\top}(P) \mid\mid}$

Для аффинно независимых попарно различных точек матрица под детерминантом всегда получается квадратной положительно определённой, а сам детерминант такой матрицы — число всегда положительное. Для аффинно зависимых точек матрица получится сингулярной (т.е. её определитель равен нулю). Ясно, что мера всегда неотрицательная и информации об ориентации мы не имеем.
С одной стороны, детерминант произведения квадратных матриц равен произведению детерминантов, с другой — детерминант транспонированной квадратной матрицы совпадает с детерминантом исходной, таким образом последняя формула сводится к формуле для точки, за исключением корня из квадрата, т.е. модуля, который мы можем опустить, с целью получить дополнительную информацию об относительной ориентации точек в пространстве.

# Алгоритм
Авторы описали алгоритм следующим образом:

Смысл алгоритма в том, что бы берём некий изначальный симплекс, постепенно достраивая его до внешней оболочки.

Опишем более подробно:
1. Возьмём стартовый симплекс из $n+1$ вершины
2. Для каждой грани создадим множество внешних точек лежащих выше неё. Будем добавлять только ещё не рассмотренные точки. 
3. Для каждой выберем самую дальнюю от неё точку $p$. Создадим множество $V$ видимых из данной точки граней и добавим изначально в него рассматриваемую грань и добавим затем в него соседей, для которых $p$ лежит выше них, дальше будем смотреть соседей соседей, и.т.д. пока не найдём все. Множество рёбер горизонта $H$ будет границей множества $V$. Далее для каждого ребра в $H$ составим новую грань из него и точки $p$. Для каждой новой грани найдём множество внешних точек. Удалим грани, которые есть в $V$ и добавим новые на их замену.
4. Будем продолжать пункт 3, пока не получим выпуклую оболочку.